In [98]:
!pip install langchain langchain-community langchain-google-genai faiss-cpu pypdf gradio python-dotenv



In [87]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.schema import Document
import gradio as gr


In [99]:
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [100]:
pdf_path = "data/CTSE_lecture_notes.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

In [101]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [102]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [103]:
vectorstore = FAISS.from_documents(chunks, embeddings)



In [104]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0.2)


In [111]:
def custom_retriever(query, k=10, threshold=0.6):
    query_embedding = embeddings.embed_query(query)
    docs_and_scores = vectorstore.similarity_search_with_score_by_vector(query_embedding, k=k)
    filtered_docs = [doc for doc, score in docs_and_scores if score <= threshold]
    return filtered_docs

In [112]:
def ask_bot(query, show_context=False):
    docs = custom_retriever(query)
    if not docs:
        return "🤖: No relevant information found with similarity", ""

    combined_context = "\n\n".join([doc.page_content for doc in docs])
    prompt = f"""
You are a helpful teaching assistant. Answer the user's question strictly based on the provided lecture notes below.
If the answer is not present in the context, respond with: "The lecture notes do not provide this information."
Format your answer clearly and concisely using bullet points or steps where appropriate and dont use ** for bold.

Lecture Notes:
\"\"\" 
{combined_context} 
\"\"\" 

Question: {query}
Answer:
"""
    response = llm.invoke(prompt)
    return "🤖: " + response.content.strip(), combined_context if show_context else ""


In [114]:
with gr.Blocks() as interface:
    gr.Markdown("## CTSE Buddy Q&A Assistant")
    gr.Markdown("Ask questions based on the lecture notes. The assistant will respond using only information from the notes.")

    with gr.Row():
        with gr.Column():
            query_input = gr.Textbox(label="Enter your question")
            show_context_chk = gr.Checkbox(label="Show matched lecture note context")
            submit_btn = gr.Button("Ask")

        with gr.Column():
            response_output = gr.Textbox(label="Answer", lines=8)
            context_output = gr.Textbox(label="Matched Context", lines=12, visible=True)

    submit_btn.click(fn=ask_bot, inputs=[query_input, show_context_chk], outputs=[response_output, context_output])

interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7876

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
